In [1]:
import cv2
import mediapipe as mp
import time

In [57]:
class FaceMashDetector():
    def __init__(self,staticMode=False,maxFace=2,minDetectionCon=0.5,minTrackCon=0.5):
        self.staticMode=staticMode
        self.maxFace=maxFace
        self.minDetectionCon=minDetectionCon
        self.minTrackCon=minTrackCon
        self.mpDraw=mp.solutions.drawing_utils
        self.mpFaceMesh=mp.solutions.face_mesh
        self.faceMesh = self.mpFaceMesh.FaceMesh(static_image_mode=self.staticMode,max_num_faces=self.maxFace,min_detection_confidence=self.minDetectionCon,min_tracking_confidence=self.minTrackCon)
        self.drawSpec=self.mpDraw.DrawingSpec(thickness=1,circle_radius=1)
    def findFaceMesh(self,img,draw=True):
        self.imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results= self.faceMesh.process(self.imgRGB)
        faces=[]
        if self.results.multi_face_landmarks:
            for faceLms in self.results.multi_face_landmarks:
                self.mpDraw.draw_landmarks(img,faceLms,self.mpFaceMesh.FACEMESH_CONTOURS,self.drawSpec,self.drawSpec) 
            face=[]
            for id,lm in enumerate(faceLms.landmark):
                ih,iw,ic=img.shape
                x,y=int(lm.x*iw),int(lm.y*ih)
                cv2.putText(img,str(id),(x,y),cv2.FONT_HERSHEY_PLAIN,0.2,(0,255,0),1)
                face.append([x,y])
            faces.append(face)
        return img,faces

In [58]:
def main():
    cap = cv2.VideoCapture(r"C:\\Users\\Shriniwas\\Desktop\\video.mp4")
    pTime=0
    detector=FaceMashDetector()
    while True:
        success,img=cap.read()
        down_width = 900
        down_height = 700
        down_points = (down_width, down_height)
        img = cv2.resize(img, down_points, interpolation= cv2.INTER_LINEAR)
        img,faces=detector.findFaceMesh(img)
        cTime=time.time()
        fps=1/(cTime-pTime)
        pTime=cTime
        cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_PLAIN,3,(255,0,255),3)
        cv2.imshow('Image',img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        #cv2.waitKey(1)
    cap.release()
    cv2.destroyAllWindows()
if __name__=='__main__':
    main()